In [1]:
# O objetivo aqui é estudar e encontrar soluções para lidar com a quantidade excessiva de NaN's do dataset.
# A figura abaixo mostra a porcentagem de NaN's no Dataset o qual queremos lidar (passe o mouse em cima!).

![alt-text](\Users\joao\Desktop\image.png "Porcentagem de NaN's das variáveis que queremos lidar")

In [2]:
import os
import pandas as pd
import AnaliseExploratoria_functions
from AnaliseExploratoria_functions import histogram_and_stats

current_path = os.getcwd()
csv_path = (current_path + '/Data/campeonatos_futebol_atualizacao.csv')

In [3]:
df = pd.read_csv(csv_path)
df

,Chutes a gol 1,Chutes a gol 2,Impedimentos 1,Impedimentos 2,Escanteios 1,Escanteios 2,Chutes fora 1,Chutes fora 2,Faltas 1,Faltas 2,...,Tiros-livres 1,Tiros-livres 2,Defesas difíceis 1,Defesas difíceis 2,Posse 1(%),Posse 2(%),Time 1,Time 2,Position 1,Position 2
0,8.0,0.0,6.0,3.0,7.0,1.0,6.0,1.0,8.0,14.0,...,NaN,NaN,NaN,NaN,77.0,23.0,Glasgow Rangers,Livingston,4-3-3,4-3-3
1,0.0,2.0,0.0,2.0,0.0,4.0,3.0,4.0,19.0,14.0,...,NaN,NaN,NaN,NaN,44.0,56.0,Ross County,St.Johnstone,3-5-2,3-5-2
2,4.0,5.0,1.0,5.0,8.0,11.0,2.0,5.0,13.0,14.0,...,NaN,NaN,NaN,NaN,47.0,53.0,Dundee FC,St. Mirren,4-3-3,3-4-1-2
3,4.0,7.0,8.0,1.0,6.0,5.0,4.0,7.0,4.0,11.0,...,NaN,NaN,NaN,NaN,27.0,73.0,Hearts,Celtic,3-4-3,4-2-3-1
4,3.0,1.0,1.0,3.0,5.0,4.0,2.0,2.0,12.0,17.0,...,NaN,NaN,NaN,NaN,58.0,42.0,Aberdeen,Dundee U.,4-4-2,3-5-1-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27711,6.0,2.0,1.0,0.0,9.0,6.0,6.0,5.0,7.0,7.0,...,NaN,NaN,NaN,NaN,52.0,48.0,Aston Villa,Blackburn,4-4-2,4-5-1
27712,6.0,0.0,4.0,3.0,4.0,2.0,8.0,2.0,11.0,9.0,...,NaN,NaN,NaN,NaN,52.0,48.0,Arsenal,Fulham,4-2-3-1,4-4-2
27713,7.0,7.0,2.0,3.0,10.0,2.0,8.0,0.0,10.0,6.0,...,NaN,NaN,NaN,NaN,55.0,45.0,Manchester Utd,Stoke,4-4-2,4-4-2
27714,6.0,6.0,1.0,2.0,4.0,3.0,4.0,6.0,5.0,3.0,...,NaN,NaN,NaN,NaN,45.0,55.0,West Ham,Manchester City,4-5-1,4-4-2


In [4]:
# primeiro precisamos tirar os outliers do dataframe. Podemos identificar os outliers.
# Podemos identificar outliers com a função "histogram_and_stats" da biblioteca AnaliseExploratoria_functions

In [5]:
print(df.columns) # todas as colunas do dataframe original

Index(['Chutes a gol 1', 'Chutes a gol 2', 'Impedimentos 1', 'Impedimentos 2',
       'Escanteios 1', 'Escanteios 2', 'Chutes fora 1', 'Chutes fora 2',
       'Faltas 1', 'Faltas 2', 'Cartões amarelos 1', 'Cartões amarelos 2',
       'Cartões vermelhos 1', 'Cartões vermelhos 2', 'Cruzamentos 1',
       'Cruzamentos 2', 'Laterais 1', 'Laterais 2', 'Chutes bloqueados 1',
       'Chutes bloqueados 2', 'Contra-ataques 1', 'Contra-ataques 2', 'Gols 1',
       'Gols 2', 'Tiro de meta 1', 'Tiro de meta 2', 'Tratamentos 1',
       'Tratamentos 2', 'Substituições 1', 'Substituições 2', 'Tiros-livres 1',
       'Tiros-livres 2', 'Defesas difíceis 1', 'Defesas difíceis 2',
       'Posse 1(%)', 'Posse 2(%)', 'Time 1', 'Time 2', 'Position 1',
       'Position 2'],
      dtype='object')


In [6]:
# Vamos selecionar apenas as colunas com valores numéricos e exibir suas estatísticas

In [7]:
for col in df.select_dtypes(include="number").columns:
    print(f'\033[1m{col}\033[0m')
    histogram_and_stats(df,col,plot=False)

Chutes a gol 1
NaN's: 1512 --|-- Ocorrências: 26204 --|-- Média: 4.77 --|-- Desv. padrão: 2.80 --|-- Mínimo: 0.00 --|-- Máximo: 90.00 --|-- 5º percentil: 1.00 --|-- 25º percentil: 3.00 --|-- 50º percentil (mediana): 4.00 --|-- 75º percentil: 6.00 --|-- 95º percentil: 10.00
--------------
Chutes a gol 2
NaN's: 1512 --|-- Ocorrências: 26204 --|-- Média: 3.85 --|-- Desv. padrão: 2.44 --|-- Mínimo: 0.00 --|-- Máximo: 80.00 --|-- 5º percentil: 1.00 --|-- 25º percentil: 2.00 --|-- 50º percentil (mediana): 4.00 --|-- 75º percentil: 5.00 --|-- 95º percentil: 8.00
--------------
Impedimentos 1
NaN's: 2774 --|-- Ocorrências: 24942 --|-- Média: 2.14 --|-- Desv. padrão: 1.75 --|-- Mínimo: 0.00 --|-- Máximo: 23.00 --|-- 5º percentil: 0.00 --|-- 25º percentil: 1.00 --|-- 50º percentil (mediana): 2.00 --|-- 75º percentil: 3.00 --|-- 95º percentil: 5.00
--------------
Impedimentos 2
NaN's: 2774 --|-- Ocorrências: 24942 --|-- Média: 1.97 --|-- Desv. padrão: 1.68 --|-- Mínimo: 0.00 --|-- Máximo: 15.00 -

In [8]:
# Analisando os valores máximos na célula acima, podemos ter uma noção de onde temos outliers.
# Nas células abaixo, usaremos alguns métodos para verificar outliers.

# Fazendo um loop pelos dados acima do percentil 99.9
df_numerico = df.select_dtypes(include='number')
for col in df_numerico.columns:
    p999 = df[col].quantile(0.999)
    outliers_col = df[df[col] > p999]

    print(f"\n📊 Coluna: {col}")
    print(f"🔎 Quantidade de outliers (> P999 = {p999:.2f}): {len(outliers_col)}")

    # Mostra os outliers da coluna (apenas as colunas relevantes)
    if not outliers_col.empty:
        display(outliers_col[[col]])
    else:
        print("✅ Nenhum outlier encontrado acima do P999.")



📊 Coluna: Chutes a gol 1
🔎 Quantidade de outliers (> P999 = 16.00): 23


,Chutes a gol 1
137,19.0
220,90.0
698,18.0
1037,19.0
1061,20.0
2101,21.0
3313,17.0
5753,17.0
6388,18.0
13493,19.0



📊 Coluna: Chutes a gol 2
🔎 Quantidade de outliers (> P999 = 14.00): 22


,Chutes a gol 2
352,16.0
605,15.0
664,22.0
718,15.0
730,16.0
1997,17.0
2101,15.0
2410,80.0
2413,30.0
9751,16.0



📊 Coluna: Impedimentos 1
🔎 Quantidade de outliers (> P999 = 10.00): 19


...

,Defesas difíceis 2
5753,14.0
6388,17.0
6715,13.0
24445,14.0



📊 Coluna: Posse 1(%)
🔎 Quantidade de outliers (> P999 = 83.00): 22


,Posse 1(%)
68,85.0
819,100.0
841,100.0
842,100.0
846,100.0
859,100.0
871,100.0
2457,100.0
6869,100.0
7001,100.0



📊 Coluna: Posse 2(%)
🔎 Quantidade de outliers (> P999 = 80.00): 25


,Posse 2(%)
75,81.0
113,83.0
571,99.0
868,100.0
2305,100.0
2841,100.0
3148,100.0
3183,100.0
3997,100.0
6837,100.0


In [9]:

# Pelo Métdo do Intervalo Quartil 

# O IQR (Intervalo Interquartil) é uma medida estatística que mostra a dispersão central dos dados, ignorando os valores extremos. Ele é muito usado para detectar outliers.
# - Q1 (1º quartil): valor que separa os 25% menores valores
# - Q3 (3º quartil): valor que separa os 75% menores valores
# - O IQR seria IQR = Q3-Q1

# Definimos o limite para outliers:
# Limite  = Q3 + 1.5 × IQR
# Qualquer valor fora desse intervalo é considerado outlier.

# Portanto, aplicando o métdo

df_numerico = df.select_dtypes(include='number')
for col in df_numerico.columns:
    q001 = df[col].quantile(0.01)
    q99 = df[col].quantile(0.99)
    iqr = q99 - q001

    limite_inferior = q001 - 1.5 * iqr
    limite_superior = q99 + 1.5 * iqr

    outliers = df[(df[col] < limite_inferior) | (df[col] > limite_superior)]

    print(f"\n📊 Coluna: {col}")
    print(f"🔎 Outliers encontrados: {len(outliers)}")

    if not outliers.empty:
        print(outliers[[col]].head())  # Mostra só os primeiros outliers
    else:
        print("✅ Nenhum outlier detectado.")



📊 Coluna: Chutes a gol 1
🔎 Outliers encontrados: 1
     Chutes a gol 1
220            90.0

📊 Coluna: Chutes a gol 2
🔎 Outliers encontrados: 2
      Chutes a gol 2
2410            80.0
2413            30.0

📊 Coluna: Impedimentos 1
🔎 Outliers encontrados: 1
       Impedimentos 1
17321            23.0

📊 Coluna: Impedimentos 2
🔎 Outliers encontrados: 0
✅ Nenhum outlier detectado.

📊 Coluna: Escanteios 1
🔎 Outliers encontrados: 0
✅ Nenhum outlier detectado.

📊 Coluna: Escanteios 2
🔎 Outliers encontrados: 0
✅ Nenhum outlier detectado.

📊 Coluna: Chutes fora 1
🔎 Outliers encontrados: 0
✅ Nenhum outlier detectado.

📊 Coluna: Chutes fora 2
🔎 Outliers encontrados: 1
       Chutes fora 2
21357          748.0

📊 Coluna: Faltas 1
🔎 Outliers encontrados: 0
✅ Nenhum outlier detectado.

📊 Coluna: Faltas 2
🔎 Outliers encontrados: 3
      Faltas 2
218       90.0
2410     180.0
2414     120.0

📊 Coluna: Cartões amarelos 1
🔎 Outliers encontrados: 0
✅ Nenhum outlier detectado.

📊 Coluna: Cartões amarel

In [10]:
# Agora, pelo métoddo dos z-scores
# O z-score mede o quão distante (em número de desvios padrão) um valor está da média da sua distribuição:
from scipy.stats import zscore

z = (x−μ)/σ

In [11]:
df_numerico = df.select_dtypes(include='number')
for col in df_numerico.columns:
    z = zscore(df[col], nan_policy='omit')  # calcula z-score, ignorando NaNs
    z_series = pd.Series(z, index=df.index)
    outliers = df[z_series.abs() > 4]

    print(f'\nOutliers na coluna "{col}":')
    print(outliers[[col]])


Outliers na coluna "Chutes a gol 1":
       Chutes a gol 1
137              19.0
220              90.0
301              16.0
698              18.0
1037             19.0
1061             20.0
1097             16.0
2071             16.0
2101             21.0
3313             17.0
5753             17.0
5775             16.0
6388             18.0
6715             16.0
6743             16.0
7019             16.0
7666             16.0
8959             16.0
9346             16.0
11250            16.0
13493            19.0
13515            19.0
16085            17.0
16928            17.0
17266            19.0
17268            18.0
17269            23.0
17336            21.0
18521            17.0
21138            18.0
21632            16.0
24812            16.0
25120            18.0
25327            16.0
25529            16.0
25728            16.0
25998            16.0
26161            18.0
26966            17.0
27366            17.0
27411            16.0
27607            16.0

Outliers na col

In [12]:
## Tendo aplicado esses métodos, conseguimos identificar alguns outliers. Os outliers estão melhores detalhados no arquivo outliers.txt
outliers = (218, 220, 423, 571, 819, 841, 842, 846, 859, 868, 871, 1257,
            1490, 1497, 2305, 2410, 2414, 2457, 2841, 3148, 3183, 3997,
            5150, 6837, 6869, 6911, 7001, 7003, 7004, 7005, 7771, 10651,
            10718, 10819, 10923, 11084, 11406, 11568, 12005, 12185, 13750,
            13812, 14283, 15532, 19611, 19616, 19622, 19652, 19655, 19827,
            21357, 22641, 22669, 23085, 23133, 24787, 24788, 24838, 24855)

df.iloc[list(outliers)]


,Chutes a gol 1,Chutes a gol 2,Impedimentos 1,Impedimentos 2,Escanteios 1,Escanteios 2,Chutes fora 1,Chutes fora 2,Faltas 1,Faltas 2,...,Tiros-livres 1,Tiros-livres 2,Defesas difíceis 1,Defesas difíceis 2,Posse 1(%),Posse 2(%),Time 1,Time 2,Position 1,Position 2
218,5.0,5.0,0.0,0.0,7.0,2.0,17.0,4.0,10.0,90.0,...,NaN,NaN,NaN,NaN,67.0,33.0,Celtic,Hamilton,4-2-3-1,4-5-1
220,90.0,3.0,0.0,0.0,5.0,5.0,10.0,5.0,13.0,14.0,...,NaN,NaN,NaN,NaN,52.0,48.0,Hearts,Livingston,4-2-3-1,4-2-3-1
423,0.0,3.0,0.0,2.0,2.0,7.0,4.0,9.0,46.0,13.0,...,NaN,NaN,NaN,NaN,51.0,49.0,Motherwell,Kilmarnock,3-5-2,4-2-3-1
571,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,1.0,99.0,Ross County,Hibernian,4-4-2,4-4-2
819,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,100.0,0.0,Celtic,Hamilton,3-5-2,4-1-4-1
841,4.0,0.0,NaN,NaN,0.0,7.0,0.0,9.0,0.0,13.0,...,NaN,NaN,NaN,NaN,100.0,0.0,Motherwell,Partick T.,4-4-2,5-3-2
842,NaN,NaN,3.0,0.0,NaN,NaN,5.0,0.0,11.0,0.0,...,NaN,NaN,NaN,NaN,100.0,0.0,St.Johnstone,Ross County,4-4-1-1,4-4-2
846,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,100.0,0.0,Dundee FC,Inverness,3-5-2,4-2-3-1
859,0.0,0.0,0.0,0.0,NaN,NaN,3.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,100.0,0.0,Dundee FC,Ross County,3-5-2,4-3-3
868,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,100.0,Ross County,Aberdeen,4-4-2,4-2-3-1


In [13]:
# Vamos eliminar esses outliers.
df_sem_outliers = df.drop(df.index[list(outliers)])
df_sem_outliers

,Chutes a gol 1,Chutes a gol 2,Impedimentos 1,Impedimentos 2,Escanteios 1,Escanteios 2,Chutes fora 1,Chutes fora 2,Faltas 1,Faltas 2,...,Tiros-livres 1,Tiros-livres 2,Defesas difíceis 1,Defesas difíceis 2,Posse 1(%),Posse 2(%),Time 1,Time 2,Position 1,Position 2
0,8.0,0.0,6.0,3.0,7.0,1.0,6.0,1.0,8.0,14.0,...,NaN,NaN,NaN,NaN,77.0,23.0,Glasgow Rangers,Livingston,4-3-3,4-3-3
1,0.0,2.0,0.0,2.0,0.0,4.0,3.0,4.0,19.0,14.0,...,NaN,NaN,NaN,NaN,44.0,56.0,Ross County,St.Johnstone,3-5-2,3-5-2
2,4.0,5.0,1.0,5.0,8.0,11.0,2.0,5.0,13.0,14.0,...,NaN,NaN,NaN,NaN,47.0,53.0,Dundee FC,St. Mirren,4-3-3,3-4-1-2
3,4.0,7.0,8.0,1.0,6.0,5.0,4.0,7.0,4.0,11.0,...,NaN,NaN,NaN,NaN,27.0,73.0,Hearts,Celtic,3-4-3,4-2-3-1
4,3.0,1.0,1.0,3.0,5.0,4.0,2.0,2.0,12.0,17.0,...,NaN,NaN,NaN,NaN,58.0,42.0,Aberdeen,Dundee U.,4-4-2,3-5-1-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27711,6.0,2.0,1.0,0.0,9.0,6.0,6.0,5.0,7.0,7.0,...,NaN,NaN,NaN,NaN,52.0,48.0,Aston Villa,Blackburn,4-4-2,4-5-1
27712,6.0,0.0,4.0,3.0,4.0,2.0,8.0,2.0,11.0,9.0,...,NaN,NaN,NaN,NaN,52.0,48.0,Arsenal,Fulham,4-2-3-1,4-4-2
27713,7.0,7.0,2.0,3.0,10.0,2.0,8.0,0.0,10.0,6.0,...,NaN,NaN,NaN,NaN,55.0,45.0,Manchester Utd,Stoke,4-4-2,4-4-2
27714,6.0,6.0,1.0,2.0,4.0,3.0,4.0,6.0,5.0,3.0,...,NaN,NaN,NaN,NaN,45.0,55.0,West Ham,Manchester City,4-5-1,4-4-2


In [14]:
for col in df.select_dtypes(include="number").columns:
    print(f'\033[1m{col}\033[0m')
    histogram_and_stats(df,col,plot=False)

Chutes a gol 1
NaN's: 1512 --|-- Ocorrências: 26204 --|-- Média: 4.77 --|-- Desv. padrão: 2.80 --|-- Mínimo: 0.00 --|-- Máximo: 90.00 --|-- 5º percentil: 1.00 --|-- 25º percentil: 3.00 --|-- 50º percentil (mediana): 4.00 --|-- 75º percentil: 6.00 --|-- 95º percentil: 10.00
--------------
Chutes a gol 2
NaN's: 1512 --|-- Ocorrências: 26204 --|-- Média: 3.85 --|-- Desv. padrão: 2.44 --|-- Mínimo: 0.00 --|-- Máximo: 80.00 --|-- 5º percentil: 1.00 --|-- 25º percentil: 2.00 --|-- 50º percentil (mediana): 4.00 --|-- 75º percentil: 5.00 --|-- 95º percentil: 8.00
--------------
Impedimentos 1
NaN's: 2774 --|-- Ocorrências: 24942 --|-- Média: 2.14 --|-- Desv. padrão: 1.75 --|-- Mínimo: 0.00 --|-- Máximo: 23.00 --|-- 5º percentil: 0.00 --|-- 25º percentil: 1.00 --|-- 50º percentil (mediana): 2.00 --|-- 75º percentil: 3.00 --|-- 95º percentil: 5.00
--------------
Impedimentos 2
NaN's: 2774 --|-- Ocorrências: 24942 --|-- Média: 1.97 --|-- Desv. padrão: 1.68 --|-- Mínimo: 0.00 --|-- Máximo: 15.00 -

In [15]:
# Vamos salvar esses dados sem os outliers.
df_sem_outliers.to_csv(current_path + '/Data/campeonatos_futebol_sem_outliers.csv',
                       index=False)